# **Exercise**
> Write a *function that can shift an `MNIST` image in any direction (left,
right, up, or down) by one pixel*. Then, for each image in the training
set, *create four shifted copies (one per direction) and add them to the
training set*. Finally, *train your best model* on this expanded training set
and **measure its accuracy on the test set**. You should observe that your
model performs even better now! This technique of artificially growing
the training set is called **data augmentation** or **training set expansion**.
    
---

### **Step 1:** Load the MNIST dataset and Split the dataset into training and test sets

We use `fetch_openml()` to load the classic MNIST dataset, which consists of 70,000 grayscale images (28×28 pixels) of handwritten digits. We also cast the labels to `uint8` to save memory and avoid type issues.

The first **60,000** images are used for training, and the remaining **10,000** for testing — this is standard for `MNIST`.

In [2]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype(np.uint8)
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

### **Step 2:** Define the shift function
This function **shifts an image by one pixel in a given direction** (`up`, `down`, `left`, or `right`). The image is reshaped to 28×28 for the operation, then flattened back to 784 (as required by scikit-learn).

In [3]:
def shift_image(image, direction):
    image = image.reshape(28, 28)
    shifted = np.zeros_like(image)

    if direction == 'up':
        shifted[:-1, :] = image[1:, :]
    elif direction == 'down':
        shifted[1:, :] = image[:-1, :]
    elif direction == 'left':
        shifted[:, :-1] = image[:, 1:]
    elif direction == 'right':
        shifted[:, 1:] = image[:, :-1]

    return shifted.reshape(784)

### **Step 3:** Apply shifting to every image and expand the dataset

We create 4 shifted copies for every training image and concatenate them with the original training set. The label (`y`) remains the same for shifted images, since we're only transforming the inputs.

In [4]:
X_augmented = [X_train]
y_augmented = [y_train]

for direction in ['up', 'down', 'left', 'right']:
    shifted_images = np.apply_along_axis(shift_image, axis=1, arr=X_train, direction=direction)
    X_augmented.append(shifted_images)
    y_augmented.append(y_train)

X_train_expanded = np.concatenate(X_augmented)
y_train_expanded = np.concatenate(y_augmented)

### **Step 5:** Train the best classifier
We use the best hyperparameters from Exercise 1 (`KNeighborsClassifier` with `n_neighbors=4` and `weights='distance'`) to train on the expanded dataset.

(**FYI**: I did not run this directly on my pc, otherwise it might explode)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance')
knn_clf.fit(X_train_expanded, y_train_expanded)

###  **Step 6:** Evaluate accuracy on the original test set
Finally, we test the model on the **original 10,000-image test set** to measure generalization. We see an improvement over Exercise 1 (which had **~97.1% accuracy**).

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

### **Expected Result**
By increasing the diversity of the training data (through shifting), the model should now generalize better and surpass the accuracy of the original KNN model. This is a practical example of **data augmentation**, a powerful technique to improve performance without collecting new data.